In [1]:
import pandas as pd
import numpy as np 
import plotly.express as ex 
import plotly.graph_objects as go

# 数据清洗&预处理

In [2]:
#加载三张数据集
user_df = pd.read_csv('databases/user—test.csv',encoding='gb2312')
order_df = pd.read_csv('databases/Order-test.csv')
consult_df = pd.read_csv('databases/consult-test.csv')

## 处理 User 表格

In [3]:
print(user_df.info(),"\n")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 870460 entries, 0 to 870459
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user_id         870460 non-null  int64  
 1   sex             62753 non-null   object 
 2   age             61970 non-null   float64
 3   country         866918 non-null  object 
 4   province_name   866918 non-null  object 
 5   city_name       866918 non-null  object 
 6   city_level      863913 non-null  object 
 7   is_member_flag  870460 non-null  int64  
 8   first_day       811277 non-null  object 
 9   last_day        870460 non-null  object 
 10  be_member_time  596011 non-null  object 
dtypes: float64(1), int64(2), object(8)
memory usage: 73.1+ MB
None 



In [4]:
# 这里 sex & age Missing value 是80w+, 这个数据集总共才87W, 缺失值总共约占数据集92%
# 直接去掉这两行
user_df = user_df.drop(['sex','age'],axis=1)

In [5]:
# 查看province_name 为缺失值有3542个，由于省名缺失，对后续分析有影响。
# 所以,直接将这三千多行数据去掉，约占总数据集的0.4%，影响不大
# user_df = user_df.dropna(subset=['province_name'])
# columnlist = user_df.columns.values
print(user_df.isna().sum())
print(user_df.shape)

user_id                0
country             3542
province_name       3542
city_name           3542
city_level          6547
is_member_flag         0
first_day          59183
last_day               0
be_member_time    274449
dtype: int64
(870460, 9)


In [6]:
#缺失city_level这些地方属于一些特殊的省直管县级市，按照这种方法，使用‘其他城市’去替换Nan
user_df['city_level'] = user_df['city_level'].fillna("其他城市")
#检测是否有重复row
print(user_df.duplicated().sum())

0


In [7]:
user_df['first_day'] = pd.to_datetime(pd.to_datetime(user_df['first_day'], errors='coerce').dt.strftime('%Y-%m-%d %H:%M:%S'))
user_df['last_day'] = pd.to_datetime(user_df['last_day'])
user_df['be_member_time'] = pd.to_datetime(user_df['be_member_time'])

In [8]:
#关于填补first_day
user_df['first_day'] = user_df['first_day'].fillna(user_df['be_member_time'])
user_df.reset_index(drop=True)

,user_id,country,province_name,city_name,city_level,is_member_flag,first_day,last_day,be_member_time
0,570009353210116,中国,山东,潍坊,三线城市,1,2023-04-03,2023-08-02,2023-04-03 19:42:00
1,570020406530107,中国,贵州,六盘水,四线城市,1,2023-03-09,2023-03-09,2023-03-09 19:52:00
2,570020458000111,中国,广东,广州,一线城市,1,2023-06-25,2023-06-29,2023-06-29 12:38:00
3,570020914800114,中国,山东,青岛,新一线城市,1,2023-06-25,2023-07-02,2022-04-20 10:22:00
4,570021004330160,中国,江西,上饶,三线城市,1,2023-05-11,2023-07-09,2022-04-22 10:17:00
...,...,...,...,...,...,...,...,...,...
870455,570072597650156,中国,重庆,重庆,新一线城市,0,2023-06-30,2023-06-30,NaT
870456,570072523900118,中国,山东,潍坊,三线城市,0,2023-06-30,2023-08-03,NaT
870457,570072538870133,中国,黑龙江,大庆,三线城市,1,2023-06-30,2023-06-30,2023-06-30 19:08:00
870458,570072510060139,中国,陕西,西安,新一线城市,1,2023-06-30,2023-06-30,2023-06-30 21:03:00


In [9]:
user_df[user_df['user_id']==570026677930135]

,user_id,country,province_name,city_name,city_level,is_member_flag,first_day,last_day,be_member_time
820504,570026677930135,中国,四川,宜宾,四线城市,1,2022-08-19 19:14:00,2023-10-30,2022-08-19 19:14:00


In [10]:
condition1 = user_df['first_day'] > user_df['be_member_time']
user_df = user_df[~condition1]
condition2 = user_df['first_day'] > user_df['last_day']
user_df = user_df[~condition2]
condition3 = user_df['last_day'] < user_df['be_member_time']
user_df = user_df[~condition3]

In [11]:
#增加续费会员的标签
user_df['renew_time'] = (user_df['be_member_time'] + pd.DateOffset(days=365))
user_df['is_renew'] = (user_df['renew_time'] <= user_df['last_day']).astype(int) #判断会员是否续费（会员到期日子早于会员最近一次登陆日期）

In [12]:
user_df['mbr_days'] = (user_df['be_member_time'] - user_df['first_day']).dt.days #普通用户到付费用户转化天数
user_df['mbr_lose_days'] = (user_df['last_day'] - user_df['be_member_time']).dt.days #会员到流失转化天数

In [13]:
#将city_level 转化为数字类型，方便后续分析
def cityLevel(level):
    if level == "一线城市" or level == "新一线城市":
        return 5
    elif level == "二线城市":
        return 4
    elif level == "三线城市":
        return 3
    elif level == "四线城市":
        return 2
    elif level == "五线城市":
        return 1
    else:
        return 0
user_df['city_cate'] = user_df['city_level'].apply(cityLevel)

In [14]:
user_df.to_csv('databases/clean_user_df.csv',encoding='utf-8')

In [15]:
user_df.head(5)

,user_id,country,province_name,city_name,city_level,is_member_flag,first_day,last_day,be_member_time,renew_time,is_renew,mbr_days,mbr_lose_days,city_cate
0,570009353210116,中国,山东,潍坊,三线城市,1,2023-04-03 00:00:00,2023-08-02,2023-04-03 19:42:00,2024-04-02 19:42:00,0,0.0,120.0,3
9,570023426180132,中国,北京,北京,一线城市,0,2023-03-04 00:00:00,2023-03-09,NaT,NaT,0,NaN,NaN,5
12,570025239650135,中国,上海,上海,一线城市,1,2023-03-08 00:00:00,2023-08-17,2023-07-15 18:25:00,2024-07-14 18:25:00,0,129.0,32.0,5
17,570026463920104,中国,黑龙江,哈尔滨,二线城市,1,2022-08-16 14:37:00,2023-07-18,2022-08-16 14:37:00,2023-08-16 14:37:00,0,0.0,335.0,4
18,570027668630107,中国,四川,成都,新一线城市,0,2023-03-25 00:00:00,2023-03-25,NaT,NaT,0,NaN,NaN,5


## 处理 Consult 表格

In [16]:
print(consult_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 662158 entries, 0 to 662157
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   consult_no          662158 non-null  int64  
 1   consult_start_time  662158 non-null  object 
 2   consult_end_time    662158 non-null  object 
 3   user_id             662158 non-null  int64  
 4   consult_round       0 non-null       float64
 5   like_level          94404 non-null   float64
dtypes: float64(2), int64(2), object(2)
memory usage: 30.3+ MB
None


In [17]:
# 删除 consult_round 这个variable 全是None 所以可以直接去除
del consult_df['consult_round']

In [18]:
consult_df['consult_start_time'] = pd.to_datetime(consult_df['consult_start_time'],errors='coerce').dt.tz_localize(None)
consult_df['consult_end_time'] = pd.to_datetime(consult_df['consult_end_time'],errors='coerce').dt.tz_localize(None)
consult_df = consult_df[consult_df['consult_start_time'] != consult_df['consult_end_time']] #删除问诊时间异常值

In [19]:
# 计算用户问诊时长
consult_df.loc[:,'duration'] = (consult_df['consult_end_time'] - consult_df['consult_start_time']).dt.seconds/60
consult_df.loc[:, 'duration'] = consult_df['duration'].round()
consult_df['consult_month'] = consult_df['consult_start_time'].dt.to_period('M') #提取问诊时间的月份

In [20]:
# 计算用户最后一次问诊的时间，日记截至2024年元旦，因为此数据集只到2023年底
consult_df['last_consult'] = (pd.to_datetime('2024-01-01') - consult_df['consult_end_time']).dt.days
consult_df['comment'] = consult_df['like_level'].notna().astype(int) #判断用户是否评价
consult_df = consult_df.reset_index(drop='True')
consult_df.to_csv("databases/clean_consult.csv",encoding='utf-8')

## 患者问诊行为特征构建

In [21]:
user_df = pd.read_csv('databases/clean_user_df.csv',encoding='utf-8')
consult_df = pd.read_csv('databases/clean_consult.csv',encoding='utf-8')
user_consult_df = pd.merge(consult_df,user_df,on='user_id',how='left')
del user_consult_df['Unnamed: 0_x']
del user_consult_df['Unnamed: 0_y']

In [22]:
def consult_calcu(df):
    if 'user_id' not in df.columns:
        raise ValueError("Varible user_id Does Not Exist")

    temp = df.groupby('user_id').agg({
        'consult_no':'count',
        'duration': 'sum',
        'comment':'sum',
        'like_level':lambda x: x[x >= 4].count(),
        'consult_month':lambda x:x.nunique(),
        'last_consult':'min'
    })
    
    #构建其他指标
    temp['consult_mcnts'] = temp['consult_no']/temp['consult_month'] #月均问诊次数
    temp['consult_mdurs'] = temp['duration']/temp['consult_month'] #月均问诊时长
    temp['consult_durs'] = temp['duration']/temp['consult_no'] #单次平均问诊时长
    temp['comment_ratio'] = temp['comment'] / temp['consult_no'] #评价率
    temp['favor_ration'] = temp['like_level'] / temp['comment'].sum() #好评率

    user_consult_stat = pd.DataFrame(df['user_id'].unique(), columns=['user_id'])

    user_consult_stat = pd.merge(user_consult_stat, temp, on='user_id', how='left')
    user_consult_stat.rename(columns={
        'consult_no':'consult_cnt',
    },inplace=True)

    return user_consult_stat

user_consult_mertics = consult_calcu(consult_df)
user_consult_mertics.head(3)

,user_id,consult_cnt,duration,comment,like_level,consult_month,last_consult,consult_mcnts,consult_mdurs,consult_durs,comment_ratio,favor_ration
0,570051393580120,172,1944.0,30,30,3,276,57.333333,648.0,11.302326,0.174419,0.000318
1,570051491040126,9,110.0,0,0,2,321,4.500000,55.0,12.222222,0.000000,0.000000
2,570050307940122,1,20.0,0,0,1,361,1.000000,20.0,20.000000,0.000000,0.000000


In [23]:
renew_user = user_consult_df[user_consult_df['is_renew'] == 1] #续费会员数据集
nonrenew_user = user_consult_df[user_consult_df['is_renew']==0] #非续费会员数据集

In [24]:
renew_user_period = renew_user[renew_user['consult_start_time'] < renew_user['renew_time']] #
renew_user_period = renew_user_period[renew_user_period['consult_start_time'] > renew_user_period['be_member_time']]

nonrenew_user_period = nonrenew_user[nonrenew_user['consult_start_time'] < nonrenew_user['last_day']]
nonrenew_user_period = nonrenew_user_period[nonrenew_user_period['consult_start_time'] > nonrenew_user_period['renew_time']]

In [25]:
def process_consult_rename(df):
    df = consult_calcu(df)
    df.drop(['consult_month','comment','like_level'],axis=1,inplace=True) 
    df = df.rename(columns={
        'consult_cnt':'consult_cnt_prd',
        'duration': 'duration_prd',
        'consult_durs':'consult_durs_prd',
        'consult_mcnts':'consult_mcnts_prd',
        'consult_mdurs':'consult_mdurs_prd',
        'comment_ratio':'comment_ratio_prd',
        'favor_ration':'favor_ration_prd',
        'last_consult':'last_consult_prd',
    })
    return df
renew_user = process_consult_rename(renew_user_period)
nonrenew_user = process_consult_rename(nonrenew_user_period)

In [26]:
mbr_period_consult = pd.concat([renew_user, nonrenew_user])

/var/folders/4q/j0ysgpnj36545bb_2_dt0tr80000gn/T/ipykernel_69318/23978604.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mbr_period_consult = pd.concat([renew_user, nonrenew_user])


In [27]:
mbr_period_consult_combined = pd.merge(user_consult_mertics, mbr_period_consult, left_on='user_id', right_on='user_id', how='left')

In [28]:
mbr_period_consult_combined.to_csv("databases/user_consult.csv",index=False)

## 处理 Order 数据集

In [29]:
print(order_df.info(),'\n')
print(f"下单用户人数：{order_df['user_id'].nunique()}")
print(f"拥有SKU数据：{order_df['sku_id'].nunique()}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95781 entries, 0 to 95780
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   order_no    95781 non-null  object
 1   user_id     95781 non-null  int64 
 2   sku_id      95781 non-null  object
 3   order_num   95781 non-null  int64 
 4   order_amt   95781 non-null  int64 
 5   order_time  95781 non-null  object
dtypes: int64(3), object(3)
memory usage: 4.4+ MB
None 

下单用户人数：29875
拥有SKU数据：3866


In [30]:
order_df['order_time'] = pd.to_datetime(order_df['order_time'],errors='coerce').dt.tz_localize(None)
order_df['order_month'] = order_df['order_time'].dt.to_period('M')
order_df['last_order'] = (order_df['order_time'].max() - order_df['order_time']).dt.days #用户最近一下消费时间

In [31]:
order_df.to_csv("databases/clean_order_df.csv",index=False)

In [32]:
order_df.head(5)

,order_no,user_id,sku_id,order_num,order_amt,order_time,order_month,last_order
0,4046120211191786934750,570067079770126,40441834000000000000000001736180,1,98,2023-05-31 21:59:59,2023-05,121
1,4046121032507397027334,570067079770126,40440250000000000000000000766282,1,98,2023-06-05 08:25:37,2023-06,117
2,4046121032185765031016,570067079770126,81714,2,27,2023-06-05 08:09:29,2023-06,117
3,4046120240510572950586,570065783590126,40441218000000000000000001146146,2,45,2023-06-01 12:25:53,2023-06,121
4,4046122828368625233386,570065783590126,40460840000000000000000002348044,1,99,2023-06-14 06:18:43,2023-06,108


In [46]:
#会员消费特征
def order_calcu(df):
    temp = df.groupby('user_id').agg({
        'order_no':'count',
        'order_amt':'sum',
        'order_month':lambda x: x.nunique(),
        'last_order':'min'
    })
    temp['order_mcnts'] = temp['order_no']/temp['order_month'] #月均订单数量
    temp['order_mamt'] = temp['order_amt']/temp['order_month'] #月均订单金额
    temp['order_pamt'] = temp['order_amt']/temp['order_no'] #单次平均问诊时长

    order_stat = pd.DataFrame(df['user_id'].unique(), columns=['user_id'])

    order_stat = pd.merge(order_stat, temp, on='user_id', how='left')
    order_stat.rename(columns={'order_no':'order_cnt'},inplace=True)

    return order_stat
    

def process_order_rename(df): 
    temp = order_calcu(df)
    temp.drop(['order_month'],axis=1,inplace=True)
    temp = temp.rename(columns={
        'order_cnt':'order_cnt_prd',
        'order_amt':'order_amt_prd',
        'order_mcnts':'order_mcnts_prd',
        'order_mamt':'order_mamt_prd',
        'order_pamt':'order_pamt_prd',
        'last_order':'last_order_prd'
    })

    return temp

user_order_metrics = order_calcu(order_df) #构建用户消费指标

### 搭建RMF

In [39]:
def RFM_func(x):
    if x == '111':
        return '重要价值客户'
    elif x == '011':
        return '重要保持客户'
    elif x == '101':
        return '重要发展客户'
    elif x == '001':
        return '重要挽留客户'
    elif x == '110':
        return '一般价值客户'
    elif x == '100':
        return '一般发展客户'
    elif x == '010':
        return '一般保持客户'
    else:
        return '一般挽留客户'

def RFM_mapping(x):
    if x == '重要价值客户':
        return 8
    elif x == '重要保持客户':
        return 7
    elif x == '重要发展客户':
        return 6
    elif x == '重要挽留客户':
        return 5
    elif x == '一般价值客户':
        return 4
    elif x == '一般发展客户':
        return 3
    elif x == '一般保持客户':
        return 2
    else:
        return 1


def RFM_calculation(df, columns):
    rfm_score = df[columns]
    rfm_score.columns = ['user_id','R','F','M']

    # Recency 最近一次消费时间 进行打分
    for i, j in enumerate(rfm_score['R']):
        if j <= 30:
            rfm_score['R'][i] = 5
        elif j <= 60:
            rfm_score['R'][i] = 4
        elif j <= 90:
            rfm_score['R'][i] = 3
        elif j <= 120:
            rfm_score['R'][i] = 2
        else:
            rfm_score['R'][i] = 1

    #F（Frequency）消费频率 进行打分
    for i, j in enumerate(rfm_score['F']):
        if j <= 1:
            rfm_score['F'][i] = 1
        elif j <= 2:
            rfm_score['F'][i] = 2
        elif j <= 3:
            rfm_score['F'][i] = 3
        elif j <= 4:
            rfm_score['F'][i] = 4
        else:
            rfm_score['F'][i] = 5


    #M（Monetary）消费金额 进行打分
    for i, j in enumerate(rfm_score['M']):
        if j <= 200:
            rfm_score['M'][i] = 1
        elif j <= 500:
            rfm_score['M'][i] = 2
        elif j <= 1000:
            rfm_score['M'][i] = 3
        elif j <= 2000:
            rfm_score['M'][i] = 4
        else:
            rfm_score['M'][i] = 5


    rfm_score['R_'] = rfm_score['R'].apply(lambda x: '1' if x >= rfm_score['R'].mean() else '0')
    rfm_score['F_'] = rfm_score['F'].apply(lambda x: '1' if x >= rfm_score['F'].mean() else '0')
    rfm_score['M_'] = rfm_score['M'].apply(lambda x: '1' if x >= rfm_score['M'].mean() else '0')
    rfm_score['RFM'] = rfm_score['R_'] + rfm_score['F_'] + rfm_score['M_']
    rfm_score['RFM_tag'] = rfm_score['RFM'].apply(RFM_func)
    rfm_score['RFM_level'] = rfm_score['RFM_tag'].apply(RFM_mapping)
    rfm_score.drop(columns=['R','F','M','R_','F_','M_','RFM'],inplace=True)
 
    return rfm_score


In [42]:
user_order_rfm = ['user_id', 'last_order', 'order_cnt', 'order_amt']
RFM_score = RFM_calculation(user_order_metrics, user_order_rfm)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

/var/folders/4q/j0ysgpnj36545bb_2_dt0tr80000gn/T/ipykernel_69318/1438849266.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm_score['M'][i] = 1
/var/folders/4q/j0ysgpnj36545bb_2_dt0tr80000gn/T/ipykernel_69318/1438849266.py:72: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this wi

In [48]:
user_order_metrics = pd.merge(user_order_metrics,RFM_score,on='user_id',how='left')

In [50]:
user_order_metrics.to_csv("databases/clean_order_df.csv",encoding='utf-8')

### 会员周期消费行为与RFM特征构建

In [51]:
user_df = pd.read_csv("databases/clean_user_df.csv")

In [53]:
order_user_df = pd.merge(order_df,user_df,on='user_id',how='left')

In [59]:
order_user_df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [72]:
renew_order = order_user_df[order_user_df['is_renew']==1]
nonrenew_order = order_user_df[order_user_df['is_renew']==0]

renew_order_period = renew_order[renew_order['order_time'] < renew_order['renew_time']] 
renew_order_period = renew_order_period[renew_order_period['order_time'] > renew_order_period['be_member_time']]

nonrenew_order_period = nonrenew_order[nonrenew_order['order_time'] < nonrenew_order['last_day']]
nonrenew_order_period = nonrenew_order_period[nonrenew_order_period['order_time'] > nonrenew_order_period['be_member_time']]

In [75]:
renew_order = process_order_rename(renew_order_period)
nonrenew_order = process_order_rename(nonrenew_order_period)
mbr_period_order = pd.concat([renew_order,nonrenew_order])
mbr_period_order = mbr_period_order[~mbr_period_order.index.duplicated()]

In [79]:
#计算会员周期内的RFM指标
mbr_period_order_columns = ['user_id','last_order_prd','order_cnt_prd','order_amt_prd']
RFM_score_prd = RFM_calculation(mbr_period_order,mbr_period_order_columns)

/var/folders/4q/j0ysgpnj36545bb_2_dt0tr80000gn/T/ipykernel_69318/1438849266.py:49: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  rfm_score['R'][i] = 3
/var/folders/4q/j0ysgpnj36545bb_2_dt0tr80000gn/T/ipykernel_69318/1438849266.py:49: Setting

In [80]:
mbr_period_order = pd.merge(mbr_period_order,RFM_score_prd,on='user_id',how='left')
mbr_period_order.rename(columns={
    'RFM_tag':'RFM_tag_prd',
    'RFM_level':'RFM_level_prd'
},inplace=True)

In [82]:
mbr_period_order_combined = pd.merge(user_order_metrics, mbr_period_order,left_on='user_id',right_on='user_id',how='left')

In [88]:
mbr_period_order_combined.to_csv("databases/user_order_behav.csv",encoding='utf-8')

## 数据集合并

In [89]:
user_beha_df = pd.read_csv("databases/clean_user_df.csv",encoding='utf-8')
order_beha_df = pd.read_csv("databases/user_order_behav.csv",encoding='utf-8')
consult_beha_df=pd.read_csv("databases/user_consult.csv",encoding='utf-8')

In [90]:
behavior_df = pd.merge(user_beha_df, order_beha_df, left_on='user_id',right_on='user_id', how='left')
behavior_df = pd.merge(behavior_df, consult_beha_df, left_on='user_id',right_on='user_id', how='left')

In [97]:
behavior_df.drop(columns=['Unnamed: 0_x','Unnamed: 0_y','country','province_name',
                 'city_name','city_level','RFM_tag','RFM_tag_prd','first_day',
                 'last_day','be_member_time','renew_time','mbr_lose_days'],inplace=True)

In [100]:
behavior_df.to_csv("databases/behavior.csv",encoding='utf-8')